In [15]:
%reload_kedro

2023-09-18 21:49:59,876 - kedro.extras.extensions.ipython - INFO - No path argument was provided. Using: /Users/gpalazzo/Desktop/dev/crypto_msc_thesis
2023-09-18 21:50:00,128 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2023-09-18 21:50:01,004 - kedro.extras.extensions.ipython - INFO - ** Kedro project crypto_thesis
2023-09-18 21:50:01,006 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2023-09-18 21:50:01,018 - kedro.extras.extensions.ipython - INFO - Registered line magic `run_viz`


In [16]:
from crypto_thesis.data_domains.modeling.lstm import _build_lstm_timestamps_seq
from pprint import pprint
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
import numpy as np
from keras.engine.sequential import Sequential
from keras.layers import LSTM, BatchNormalization, Dense, Dropout, Activation
from keras.models import Sequential
from keras.optimizers import Adam
from keras.regularizers import l2
from crypto_thesis.utils import optimize_params
from sklearn.model_selection import GridSearchCV

## Base

In [17]:
TARGET_COL = ["label"]
INDEX_COL = "window_nbr"

In [18]:
mt_train_multic = catalog.load("master_table_train_multic")
seq_length = catalog.load("params:lstm_timestamp_seq_length")

2023-09-18 21:50:01,123 - kedro.io.data_catalog - INFO - Loading data from `master_table_train_multic` (ParquetDataSet)...
2023-09-18 21:50:01,299 - kedro.io.data_catalog - INFO - Loading data from `params:lstm_timestamp_seq_length` (MemoryDataSet)...


## LSTM

In [19]:
# set numpy seed
np.random.seed(0)
# set tensorflow seed
tf.random.set_seed(0)

tf.keras.utils.disable_interactive_logging()

In [20]:
master_table_train = mt_train_multic.set_index(INDEX_COL)
X_train, y_train = master_table_train.drop(columns=TARGET_COL), master_table_train[TARGET_COL]

X_train_scaled_seq, y_train_scaled_seq = _build_lstm_timestamps_seq(X=X_train,
                                                                    y=y_train,
                                                                    seq_length=seq_length)

In [21]:
def create_model_lstm(optimizer: str) -> Sequential:

    # parameters
    LAYERS = [20, 20, 20, 1] #[10, 10, 10, 1]                # number of units in hidden and output layers
    N = X_train_scaled_seq.shape[2]                 # number of features
    LR = 0.0005 #0.0005                            # learning rate of the gradient descent
    LAMBD = 0.005 #0.001                         # lambda in L2 regularizaion
    DP = 0.0 #0.0                             # dropout rate
    RDP = 0.0 #0.0                            # recurrent dropout rate

    # model
    model = Sequential()
    model.add(LSTM(
        input_shape=(seq_length, N),
        units=LAYERS[0],
        activation='tanh',
        recurrent_activation='hard_sigmoid',
        kernel_regularizer=l2(LAMBD),
        recurrent_regularizer=l2(LAMBD),
        dropout=DP,
        recurrent_dropout=RDP,
        return_sequences=True,
        return_state=False,
        stateful=False,
        unroll=False
                ))
    model.add(BatchNormalization())
    model.add(LSTM(
        units=LAYERS[1],
        activation='tanh',
        recurrent_activation='hard_sigmoid',
        kernel_regularizer=l2(LAMBD),
        recurrent_regularizer=l2(LAMBD),
        dropout=DP,
        recurrent_dropout=RDP,
        return_sequences=True,
        return_state=False,
        stateful=False,
        unroll=False
                ))
    model.add(BatchNormalization())
    model.add(LSTM(
        units=LAYERS[2],
        activation='tanh',
        recurrent_activation='hard_sigmoid',
        kernel_regularizer=l2(LAMBD),
        recurrent_regularizer=l2(LAMBD),
        dropout=DP,
        recurrent_dropout=RDP,
        return_sequences=False,
        return_state=False,
        stateful=False,
        unroll=False
                ))
    model.add(BatchNormalization())
    model.add(Dense(
        units=LAYERS[3],
        activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


lstm_model_params = {
    'batch_size': [4, 16],
    'optimizer': ['SGD', 'RMSprop', 'Adam']
    }
model = KerasClassifier(build_fn=create_model_lstm, epochs=10, verbose=1)

grid = GridSearchCV(estimator=model, param_grid=lstm_model_params, n_jobs=1, cv=3)
grid_result = grid.fit(X_train, y_train)

print("Best accuracy of: %f using %s" % (grid_result.best_score_, 
                                         grid_result.best_params_))

2023-09-18 21:50:01,795 - absl - INFO - Epoch 1/10
2023-09-18 21:50:02,054 - absl - INFO - Epoch 1/10
2023-09-18 21:50:02,353 - absl - INFO - Epoch 1/10
2023-09-18 21:50:02,613 - absl - INFO - Epoch 1/10
2023-09-18 21:50:02,873 - absl - INFO - Epoch 1/10
2023-09-18 21:50:03,122 - absl - INFO - Epoch 1/10
2023-09-18 21:50:03,391 - absl - INFO - Epoch 1/10
2023-09-18 21:50:03,654 - absl - INFO - Epoch 1/10
2023-09-18 21:50:03,917 - absl - INFO - Epoch 1/10
2023-09-18 21:50:04,189 - absl - INFO - Epoch 1/10
2023-09-18 21:50:04,442 - absl - INFO - Epoch 1/10
2023-09-18 21:50:04,692 - absl - INFO - Epoch 1/10
2023-09-18 21:50:04,940 - absl - INFO - Epoch 1/10
2023-09-18 21:50:05,189 - absl - INFO - Epoch 1/10
2023-09-18 21:50:05,810 - absl - INFO - Epoch 1/10
2023-09-18 21:50:06,060 - absl - INFO - Epoch 1/10
2023-09-18 21:50:06,320 - absl - INFO - Epoch 1/10
2023-09-18 21:50:06,572 - absl - INFO - Epoch 1/10


ValueError: 
All the 18 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/keras/wrappers/scikit_learn.py", line 248, in fit
    return super().fit(x, y, **kwargs)
  File "/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/keras/wrappers/scikit_learn.py", line 175, in fit
    history = self.model.fit(x, y, **fit_args)
  File "/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
